<a href="https://colab.research.google.com/github/buzhangjiuzhou/DLfinal/blob/master/bert_yelpBinary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 导入包
# transformer提供了一些训练好的模型，可以很方便的使用。
!pip install transformers
import tensorflow as tf
import pandas as pd
import os
import tqdm
import random 
import math
# 使用分类的模型，增加了一个head用于分类。
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
% matplotlib inline

tf.__version__

     |████████████████████████████████| 757kB 9.0MB/s 
     |████████████████████████████████| 1.1MB 43.6MB/s 
     |████████████████████████████████| 890kB 61.6MB/s 
     |████████████████████████████████| 3.0MB 50.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f0caead7b477ad9651657e769d0f82582c1338903fbef988669ec33a1112c2ff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


'2.2.0'

In [2]:
# 导入现成的模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.summary()
model.config

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [3]:
# 数据链接，可以在 https://course.fast.ai/datasets 找到。
# agnews数据集，类别，标题，描述。
ag_url = 'https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz'

In [4]:
# 下载数据，并指定此时数据集的目录
ag_zip_file = tf.keras.utils.get_file(origin=ag_url,fname='yelp_review_polarity_csv.tgz', extract=True)
base_dir = os.path.join(os.path.dirname(ag_zip_file), 'yelp_review_polarity_csv')
os.listdir(base_dir)

166379520/166373201 [==============================] - 6s 0us/step


['readme.txt', 'test.csv', 'train.csv']

In [5]:
# 读取数据
# base_dir = ''
train = pd.read_csv(os.path.join(base_dir, 'train.csv'), header=None)
print(len(train))
print(train.head())
train = train.sample(n=50000)
print(len(train))
test = pd.read_csv(os.path.join(base_dir, 'test.csv'), header=None)
print(len(test))
print(test.head())
# f = open(os.path.join(base_dir, 'classes.txt'))
# classes = f.readlines()
# classes = [s.strip() for s in classes]
# print(classes)

560000
   0                                                  1
0  1  Unfortunately, the frustration of being Dr. Go...
1  2  Been going to Dr. Goldberg for over 10 years. ...
2  1  I don't know what Dr. Goldberg was like before...
3  1  I'm writing this review to give you a heads up...
4  2  All the food is great here. But the best thing...
50000
38000
   0                                                  1
0  2  Contrary to other reviews, I have zero complai...
1  1  Last summer I had an appointment to get new ti...
2  2  Friendly staff, same starbucks fair you get an...
3  1  The food is good. Unfortunately the service is...
4  2  Even when we didn't have a car Filene's Baseme...


In [6]:
# transformers自带的tokenizer中的encoder会把一段文本进行编码，然后增加上CLS和SEP，其中CLS的id是101，SEP的编码是102,PAD是0。
# 所以  a   dog   is  not   a   table
# [cls]  a   dog   is  not   a   table  [sep]
# 101   1037  3899  2003 2025  1037  2795   102
# %pprint #让列表横过来，好看一些。
tokenizer.encode(text='a dog is not a table', padding='max_length',max_length=512)[:20]

Pretty printing has been turned OFF


[101, 1037, 3899, 2003, 2025, 1037, 2795, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
# 预处理数据
# 按照上面的例子把数据集中的文本进行处理，并且得到对应的labels。
max_length = 512
# 把太长的句子给截断，防止后面出问题。
max_length_temp = math.floor(max_length * 0.9)
for i in tqdm.notebook.trange(train.shape[0]):
  if len(train[1][train[1].index[i]]) > max_length_temp:
    train[1][train[1].index[i]] = train[1][train[1].index[i]][0 : max_length_temp]
for i in tqdm.notebook.trange(test.shape[0]):
  if len(test[1][i]) > max_length_temp:
    test[1][i] = test[1][i][0 : max_length_temp]
train_ids = [tokenizer.encode(text=sent, padding='max_length', max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(train[1])]
test_ids = [tokenizer.encode(text=sent, padding='max_length',max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(test[1])]
train_labels = train[0].values - 1
test_labels = test[0].values - 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [9]:
# 把数据转成tensorflow张量
# train_ids是tf.Tensor组成得列表，所以用concat组合一下就行
train_ids = tf.concat(train_ids, 0)
# 把train_mask初始化为1，然后把train_ids等于0（PAD的部分）对应的值赋为0
train_mask = tf.ones(train_ids.shape)
train_mask = tf.where(tf.math.greater(train_ids, 0), train_mask, 0)
# labels本身是numpy数组，转为tf.Tensor
train_labels = tf.convert_to_tensor(train_labels)

# 测试集的处理同理test
test_ids = tf.concat(test_ids, 0)
test_mask = tf.ones(test_ids.shape)
test_mask = tf.where(tf.math.greater(test_ids, 0), test_mask, 0)
test_labels = tf.convert_to_tensor(test_labels)

print(train_ids[0])
print(train_mask[0])
print(train_labels[0])

tf.Tensor(
[  101 25391  3461  2023  2026  1008  1008  1008  1008  2023  2026  1008
  1008  1008  1008  1006  2000  2404  2009  2004 24665 20113  2135  2004
  5796 11697  2358 10354  7088  2515  1007  1032  1050  1032 24978 11124
 13453  2295  1010  2023  2003  2026  5440 10704  1005  1055  2398  2091
  1012  1045  2031  2042  2000  2055  1017  2060  3924  1998  2028  3310
  2485  1010  1998  1996  2060  2048  2991  2061  2460  1045  2123  1005
  1056  2130  2215  2000  2831  2055  2068  4902  1010  2061   999  1032
  1050  1032 13221  2100  2767  1998  1045  6976  2023  3327 10704  1005
  1055  2138  1015  1012  2009  2003  2485  1998  1016  1012  2009  2003
  4830  5968  1012  1045  2253  2006  1037  9610  7712 18003  2050  5926
  2005  2055  1017  2972  2706  1998  2000  2022  7481   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0    

In [10]:
# 训练参数
epochs = 1
batch_size = 4
validation_rate = 0.1

In [11]:
# 训练
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [12]:
# 模型训练
model.fit(x=[train_ids, train_mask], 
     y=train_labels, 
     batch_size=batch_size, 
     epochs=epochs, 
     verbose=1, 
     callbacks=None,
     validation_split=validation_rate, 
     validation_data=None, 
     shuffle=True)

11250/11250 [==============================] - 3438s 306ms/step - loss: 0.3004 - accuracy: 0.8883 - val_loss: 0.2503 - val_accuracy: 0.9180


In [13]:
# 模型测试
model.evaluate(x=[test_ids, test_mask],
        y=test_labels, 
        batch_size=4, 
        verbose=1)

9500/9500 [==============================] - 872s 92ms/step - loss: 0.2556 - accuracy: 0.9155


[0.2556469142436981, 0.9154736995697021]